# Bell State Stabilization of Superconducting Circuits with Real-time Feedback
This script, based on [zhinst-qcodes](https://github.com/zhinst/zhinst-qcodes), demonstrates how to realize **Bell state stabilization** using the Lookup Table Decoder of the PQSC. For this purpose, we detect arbitrary errors on a pair of entangled data qubits in a Bell state by reading out another pair of ancilla qubits using UHFQA. The readout results are forwarded to the PQSC for further processing and generation of feedback decision. The decision is then communicated to the HDAWGs and used for waveform selection to send out the necessary control signals to restore both the data qubits and ancilla qubits back to their original states. The concept is discussed in detail in the relative [blog post](https://blogs.zhinst.com/bahadir/2021/08/16/bell-state-stabilization-of-superconducting-qubits-with-real-time-feedback/).

Let’s dive in now to see how it works! 



Copyright (C) 2021 Zurich Instruments
This software may be modified and distributed under the terms of the MIT license. See the LICENSE file for details.

## Table of Contents:
* [1. Experimental Setup](#setup)
* [2. Import Packages](#import)
* [3. Initialization](#initialization)
* [4. Readout Parameters](#readout)
* [5. PQSC Configuration](#pqsc_config)
* [6. HDAWG Configuration](#hdawg_config)
    * [6.1 Configuration of HDAWG 1 AWG 0 for Control of Data Qubit D<sub>1</sub>](#hdawg1_awg0_config)
    * [6.2 Configuration of HDAWG 2 AWG 0 for Control of Data Qubit D<sub>2</sub>](#hdawg2_awg0_config)
    * [6.3 Configuration of HDAWG 1 AWG 1 for Control of Z-Ancilla](#hdawg1_awg1_config)
    * [6.4 Configuration of HDAWG 2 AWG 1 for Control of X-Ancilla](#hdawg2_awg1_config)
* [7. UHFQA Configuration](#uhfqa_config)
* [8. Main Program](#main)

## 1. Experimental Setup: <a class="anchor" id="setup"></a>
Here is the required experimental setup, including all the wiring.

![The setup](experimental_setup.svg)

The assignments of the devices for certain tasks are as follows:

![Bell state stabilization](channel_assignments.svg)

### 2. Import Packages <a class="anchor" id="import"></a>

Import zhinst-qcodes drivers

In [1]:
import zhinst.qcodes as ziqc

Import other external packages

In [2]:
import numpy as np
import textwrap
import itertools

### 3. Initialization <a class="anchor" id="initialization"></a>

* Set the data server host address
* The serial numbers of the instruments to run the script with

In [3]:
# Dataserver host
dataserver_host = "localhost"

# Instrument serials
serial_hdawg_1 = "dev8030"
serial_hdawg_2 = "dev8138"
serial_uhfqa = "dev2445"
serial_pqsc = "dev10003"

* Connect the instruments to data server and initialize their drivers

In [4]:
# Connect the instruments to data server
hdawg_1 = ziqc.HDAWG("hdawg1", serial_hdawg_1, interface="1gbe", host=dataserver_host)
hdawg_2 = ziqc.HDAWG("hdawg2", serial_hdawg_2, interface="1gbe", host=dataserver_host)
uhfqa = ziqc.UHFQA("uhfqa", serial_uhfqa, interface="1gbe", host=dataserver_host)
pqsc = ziqc.PQSC("pqsc", serial_pqsc, interface="1gbe", host=dataserver_host)

Successfully connected to data server at localhost:8004 api version: 6
Successfully connected to device DEV8030 on interface 1GBE
Connected to: Zurich Instruments HDAWG (serial:dev8030, firmware:67687) in 9.95s
Successfully connected to data server at localhost:8004 api version: 6
Successfully connected to device DEV8138 on interface 1GBE
Connected to: Zurich Instruments HDAWG (serial:dev8138, firmware:67687) in 9.26s
Successfully connected to data server at localhost:8004 api version: 6
Successfully connected to device DEV2445 on interface 1GBE
Connected to: Zurich Instruments UHFQA (serial:dev2445, firmware:67674) in 9.75s
Successfully connected to data server at localhost:8004 api version: 6
Successfully connected to device DEV10003 on interface 1GBE
Connected to: Zurich Instruments PQSC (serial:dev10003, firmware:67687) in 4.53s


* To create a base configuration, load factory preset to all devices

In [5]:
# Load factory default settings to all devices
hdawg_1.factory_reset()
hdawg_2.factory_reset()
uhfqa.factory_reset()
pqsc.factory_reset()

### 4. Readout Parameters <a class="anchor" id="readout"></a>

Necessary parameters to be defined for multiplexed readout of 2 ancilla qubits and storing the results in the register bank of PQSC are:
* Readout frequencies 
* Additional phase offset added to I and Q signals for simulated qubit responses
* Pulse length of the readout tones
* Number of samples for the raising/falling part of the gaussian flattop pulses
* Sample rate of UHFQA
* Trigger period sent out by the PQSC and feedback processing time. Note that this should be long enough to allow feedback processing to be completed.
* Register address in the PQSC register bank to store readout results

In [6]:
# Readout frequencies
frequencies = np.array([10, 31.5]) * 1e6
# Additional phase offset added to I and Q signals for simulated qubit responses
phases = [0, 180]
# Pulse lenght of the readout tones
readout_pulse_length = 1024
# Number of samples for the raising/falling part of the gaussian flat top pulses
rise_fall_length = 100
# Sample rate of UHFQA
SR_UHFQA = 1.8e9
# Trigger period sent out from PQSC and feedback processing time (2us should be enough)
trigger_period = 2e-6
# Select register address in the PQSC to store readout results
register_num = 1

Other dependent variables:
* Waiting time for the UHFQA to allow feedback data processing after receving the start trigger from the PQSC
* Number of readout channels to use for readout of ancilla qubits (equal to the number of ancilla qubits coupled to the same feedline).
* Number of readouts (depends on the number of ancilla qubits and number of states we are simulating)

In [7]:
# Waiting time for the UHFQA after receving the start trigger
# Divide to 16, round down, and multiply again with 16 to obtain correct waveform granularity for UHFQA
wait_samples_uhfqa = int((trigger_period * SR_UHFQA - readout_pulse_length) // 16) * 16
# Number of readout channels
num_readout_channels = len(frequencies)
# Number of times the readout is performed
num_readouts = len(phases) ** num_readout_channels

### 5. PQSC Configuration <a class="anchor" id="pqsc_config"></a>

* Configure the PQSC to use external reference clock
* Wait until the reference clock is locked

In [8]:
# Use external reference clock.
pqsc.ref_clock("external")
# Wait until locking is succesfull (30 seconds timeout)
pqsc.check_ref_clock(blocking=True, timeout=30)

* Port numbers where HDAWGs are connected to: [HDAWG1, HDAWG2]

In [9]:
# ZSync output ports to the receiver HDAWGs 
zsync_ports = [0, 10]

Settings related to execution engine:
* Number of triggers to be sent out. It must be equal to the number of readouts
* Trigger period sent out from PQSC and feedback process time. It is already set above together with other readout parameters. It must be long enough to allow feedback data processing.

In [10]:
# How many triggers to send to start synchronized actions on the HDAWG and the UHFQA
pqsc.repetitions(num_readouts)
# How long to wait for feedback to arrive from UHFQA
pqsc.holdoff(trigger_period)

**Settings related to condition unit (LUT decoder)**

![PQSC_lut_decoder_diagram_final.svg](PQSC_lut_decoder_diagram_final.svg)

Configuration of Source Register Selector:
* The source register selector is used to reduce the large set of readout registers to an address word of 16 bits. This address word serves as the index for look-up table to access its entries.
* The user can select 16 sources by indicating register numbers and indices of the desired bit inside of each register. 
* **Important note**:  If an address bit is not configured, it is automatically set to register 0, index 0. Therefore, it is better not to use Register 0 Index 0 to store any qubit readout result. Otherwise, the adress values will be wrong.
* In the code below the Source Register Selector is configured such that:
    * Register 2, Bit 1 of Readout Register Bank → Bit 0 of Address Word
    * Register 2, Bit 2 of Readout Register Bank → Bit 1 of Address Word

In [11]:
# Configure source register selector
pqsc.feedback.decoder.lut.sources[0].register(register_num)
pqsc.feedback.decoder.lut.sources[0].index(0)
pqsc.feedback.decoder.lut.sources[1].register(register_num)
pqsc.feedback.decoder.lut.sources[1].index(1)

* Generating a look up table and programing the LUT decoder:
     * The tables are programmed with an array of maximum size of 2<sup>16</sup> bytes. If the number of address bits that will be used is smaller than 16, it is possible to program the condition unit with an array of smaller size.
     * Since we have two ancilla qubits, there are a total of 2<sup>2</sup> = 4 possible combinations of the readout results. Hence, it is enough for our look-up table to have 4 entries. 

In [12]:
# Generate a look up table and program LUT decoder
lut_length = 2 ** num_readout_channels
lut = range(lut_length)
lut = np.array(lut).astype(np.uint32)
pqsc.feedback.decoder.lut.tables0(lut)
print(f"Look-tup table content: {lut}")

Look-tup table content: [0 1 2 3]


* Configure table output selector for each port to select which of the four look up tables should be forwarded to the output
* Enable condition unit output forwarding for each port connected to the receving HDAWGs

In [13]:
# Configure table output selector for each port
for port in zsync_ports:
    pqsc.zsyncs[port].output.decoder.source(0)

# Enable condition unit output forwarding for each port
for port in zsync_ports:
    pqsc.zsyncs[port].output.decoder.enable(1)

### 6. HDAWG Configuration <a class="anchor" id="hdawg_config"></a>

* Configure the HDAWGs to connect to the PQSC
* Assign the AWG cores as shown in [Experimental Setup](#setup)

In [14]:
# Connect HDAWGs to the PQSC
hdawg_1.enable_qccs_mode()
hdawg_2.enable_qccs_mode()

# Assign the AWG cores to control the qubits acording to setup
awg_D1 = hdawg_1.awgs[0]
awg_D2 = hdawg_2.awgs[0]
awg_AZ = hdawg_1.awgs[1]
awg_AX = hdawg_2.awgs[1]

* Common sequence program to use with all AWG Cores of all HDAWGs. It waits for starting trigger and feedback trigger from PQSC  and plays the correct waveform depending on the feedback data

In [15]:
# Sequence program
hdawg_program = """\
//Create one waveform
wave w1 = ones($param1$);
assignWaveIndex(w1,0);

repeat($param2$) {
    //Start trigger
    waitZSyncTrigger();
        
    //Feedback trigger
    waitZSyncTrigger();
    
    //Read feedback data and execute command table entry
    //depending on feedback data to play the correct waveform
    playWaveZSync(ZSYNC_DATA_PQSC_DECODER);

}
"""

Whenever there is a bit-flip error on one of the data qubits, we correct it by sending a R<sub>x</sub><sup>π</sup> pulse from AWG core 0 of HDAWG 1 to data qubit D<sub>1</sub>. Phase-flip errors, on the other hand, are corrected by sending a R<sub>z</sub><sup>π</sup> pulse from AWG core 0 of HDAWG 2 to data qubit D<sub>2</sub>.

#### 6.1. Configuration of HDAWG 1 AWG 0 for Control of Data Qubit D<sub>1</sub><a class="anchor" id="hdawg1_awg0_config"></a>

* Turn on the output channels
* Run the AWG in single mode

In [16]:
# Enable outputs
awg_D1.outputs(["on", "on"])
# Disable rerun
awg_D1.single(True)

* Configure *mask*, *shift* and *offset* nodes to select portion of interest in the feedback data as described in the [PQSC User Manual](https://docs.zhinst.com/pdf/ziPQSC_UserManual.pdf).

In [17]:
# Configure the DEMUX
awg_D1.zsync_decoder_mask(3)
awg_D1.zsync_decoder_shift(0)
awg_D1.zsync_decoder_offset(0)

* Setting the sequence parameters and compiling:
    * Use **Custom** sequence type to upload the sequence program.
    * Use **ZSync Trigger** mode to enable HDAWG to receive ZSync triggers
    * Use **custom_params** argument to replace the parameters **$param1$**, **$param2$** in the sequence program with the correct numbers

In [18]:
# Set sequence parameters
awg_D1.set_sequence_params(
    sequence_type="Custom",
    program=hdawg_program,
    trigger_mode="ZSync Trigger",
    custom_params=[1024, num_readouts],
)

# Upload sequence program to device and compile it
awg_D1.compile()

* Configure the command table to assign the correct waveform to each table entry.
* For demonstration purposes, we represent the feedback pulses as follows:
    * R<sub>x</sub><sup>π</sup> → Rectangular pulse with amplitude 1.0
    * No feedback action (identity I) → Rectangular pulse of amplitude 0.5

In [19]:
# Command table
awg_D1_ct = []
awg_D1_ct.append(
    {
        "index": 0,
        "waveform": {"index": 0},
        "amplitude0": {"value": 0.5, "increment": False},
        "amplitude1": {"value": 0.5, "increment": False},
    }
)
awg_D1_ct.append(
    {
        "index": 1,
        "waveform": {"index": 0},
        "amplitude0": {"value": 1, "increment": False},
        "amplitude1": {"value": 1, "increment": False},
    }
)
awg_D1_ct.append(
    {
        "index": 2,
        "waveform": {"index": 0},
        "amplitude0": {"value": 0.5, "increment": False},
        "amplitude1": {"value": 0.5, "increment": False},
    }
)
awg_D1_ct.append(
    {
        "index": 3,
        "waveform": {"index": 0},
        "amplitude0": {"value": 1, "increment": False},
        "amplitude1": {"value": 1, "increment": False},
    }
)

awg_D1.ct.load(awg_D1_ct)

#### 6.2. Configuration of HDAWG 2 AWG 0 for Control of Data Qubit D<sub>2</sub> <a class="anchor" id="hdawg2_awg0_config"></a>

* Turn on the output channels
* Run the AWG in single mode

In [20]:
# Enable outputs
awg_D2.outputs(["on", "on"])
# Disable rerun
awg_D2.single(True)

* Configure *mask*, *shift* and *offset* nodes to select portion of interest in the feedback data as described in the [PQSC User Manual](https://docs.zhinst.com/pdf/ziPQSC_UserManual.pdf).

In [21]:
# Configure the DEMUX
awg_D2.zsync_decoder_mask(3)
awg_D2.zsync_decoder_shift(0)
awg_D2.zsync_decoder_offset(0)

* Setting the sequence parameters and compiling:
    * Use **Custom** sequence type to upload the sequence program.
    * Use **ZSync Trigger** mode to enable HDAWG to receive ZSync triggers
    * Use **custom_params** argument to replace the parameters **$param1$**, **$param2$** in the sequence program with the correct numbers

In [22]:
# Set sequence parameters
awg_D2.set_sequence_params(
    sequence_type="Custom",
    program=hdawg_program,
    trigger_mode="ZSync Trigger",
    custom_params=[1024, num_readouts],
)

# Upload sequence program to device and compile it
awg_D2.compile()

* Configure the command table to assign the correct waveform to each table entry.
* For demonstration purposes, we represent the feedback pulses as follows:
    * R<sub>z</sub><sup>π</sup> → Rectangular pulse with amplitude 1.0
    * No feedback action (identity I) → Rectangular pulse of amplitude 0.5

In [23]:
# Command table
awg_D2_ct = []
awg_D2_ct.append(
    {
        "index": 0,
        "waveform": {"index": 0},
        "amplitude0": {"value": 0.5, "increment": False},
        "amplitude1": {"value": 0.5, "increment": False},
    }
)
awg_D2_ct.append(
    {
        "index": 1,
        "waveform": {"index": 0},
        "amplitude0": {"value": 0.5, "increment": False},
        "amplitude1": {"value": 0.5, "increment": False},
    }
)
awg_D2_ct.append(
    {
        "index": 2,
        "waveform": {"index": 0},
        "amplitude0": {"value": -1, "increment": False},
        "amplitude1": {"value": -1, "increment": False},
    }
)
awg_D2_ct.append(
    {
        "index": 3,
        "waveform": {"index": 0},
        "amplitude0": {"value": -1, "increment": False},
        "amplitude1": {"value": -1, "increment": False},
    }
)

awg_D2.ct.load(awg_D2_ct)

#### 6.3. Configuration of HDAWG 1 AWG 1 for Control of  Z-Ancilla <a class="anchor" id="hdawg1_awg1_config"></a>

* Turn on the output channels
* Run the AWG in single mode

In [24]:
# Enable outputs
awg_AZ.outputs(["on", "on"])
# Disable rerun
awg_AZ.single(True)

* Configure *mask*, *shift* and *offset* nodes to select portion of interest in the feedback data as described in the [PQSC User Manual](https://docs.zhinst.com/pdf/ziPQSC_UserManual.pdf).

In [25]:
# Configure the DEMUX
awg_AZ.zsync_decoder_mask(3)
awg_AZ.zsync_decoder_shift(0)
awg_AZ.zsync_decoder_offset(0)

* Setting the sequence parameters and compiling:
    * Use **Custom** sequence type to upload the sequence program.
    * Use **ZSync Trigger** mode to enable HDAWG to receive ZSync triggers
    * Use **custom_params** argument to replace the parameters **$param1$**, **$param2$** in the sequence program with the correct numbers

In [26]:
# Set sequence parameters
awg_AZ.set_sequence_params(
    sequence_type="Custom",
    program=hdawg_program,
    trigger_mode="ZSync Trigger",
    custom_params=[1024, num_readouts],
)

# Upload sequence program to device and compile it
awg_AZ.compile()

* Configure the command table to assign the correct waveform to each table entry.
* For demonstration purposes, we represent the feedback pulses as follows:
    * R<sub>x</sub><sup>π</sup> → Rectangular pulse with amplitude 1.0
    * No feedback action (identity I) → Rectangular pulse of amplitude 0.5

In [27]:
# Command table
awg_AZ_ct = []
awg_AZ_ct.append(
    {
        "index": 0,
        "waveform": {"index": 0},
        "amplitude0": {"value": 0.5, "increment": False},
        "amplitude1": {"value": 0.5, "increment": False},
    }
)
awg_AZ_ct.append(
    {
        "index": 1,
        "waveform": {"index": 0},
        "amplitude0": {"value": 1, "increment": False},
        "amplitude1": {"value": 1, "increment": False},
    }
)
awg_AZ_ct.append(
    {
        "index": 2,
        "waveform": {"index": 0},
        "amplitude0": {"value": 0.5, "increment": False},
        "amplitude1": {"value": 0.5, "increment": False},
    }
)
awg_AZ_ct.append(
    {
        "index": 3,
        "waveform": {"index": 0},
        "amplitude0": {"value": 1, "increment": False},
        "amplitude1": {"value": 1, "increment": False},
    }
)

awg_AZ.ct.load(awg_AZ_ct)

#### 6.4. Configuration of HDAWG 2 AWG 1 for Control of  X-Ancilla <a class="anchor" id="hdawg2_awg1_config"></a>

* Turn on the output channels
* Run the AWG in single mode

In [28]:
# Enable outputs
awg_AX.outputs(["on", "on"])
# Disable rerun
awg_AX.single(True)

* Configure *mask*, *shift* and *offset* nodes to select portion of interest in the feedback data as described in the [PQSC User Manual](https://docs.zhinst.com/pdf/ziPQSC_UserManual.pdf).

In [29]:
# Configure the DEMUX
awg_AX.zsync_decoder_mask(3)
awg_AX.zsync_decoder_shift(0)
awg_AX.zsync_decoder_offset(0)

* Setting the sequence parameters and compiling:
    * Use **Custom** sequence type to upload the sequence program.
    * Use **ZSync Trigger** mode to enable HDAWG to receive ZSync triggers
    * Use **custom_params** argument to replace the parameters **$param1$**, **$param2$** in the sequence program with the correct numbers

In [30]:
# Set sequence parameters
awg_AX.set_sequence_params(
    sequence_type="Custom",
    program=hdawg_program,
    trigger_mode="ZSync Trigger",
    custom_params=[1024, num_readouts],
)

# Upload sequence program to device and compile it
awg_AX.compile()

* Configure the command table to assign the correct waveform to each table entry.
* For demonstration purposes, we represent the feedback pulses as follows:
    * R<sub>x</sub><sup>π</sup> → Rectangular pulse with amplitude 1.0
    * No feedback action (identity I) → Rectangular pulse of amplitude 0.5

In [31]:
# Command table
awg_AX_ct = []
awg_AX_ct.append(
    {
        "index": 0,
        "waveform": {"index": 0},
        "amplitude0": {"value": 0.5, "increment": False},
        "amplitude1": {"value": 0.5, "increment": False},
    }
)
awg_AX_ct.append(
    {
        "index": 1,
        "waveform": {"index": 0},
        "amplitude0": {"value": 0.5, "increment": False},
        "amplitude1": {"value": 0.5, "increment": False},
    }
)
awg_AX_ct.append(
    {
        "index": 2,
        "waveform": {"index": 0},
        "amplitude0": {"value": 1, "increment": False},
        "amplitude1": {"value": 1, "increment": False},
    }
)
awg_AX_ct.append(
    {
        "index": 3,
        "waveform": {"index": 0},
        "amplitude0": {"value": 1, "increment": False},
        "amplitude1": {"value": 1, "increment": False},
    }
)

awg_AX.ct.load(awg_AX_ct)

### 7. UHFQA Configuration <a class="anchor" id="uhfqa_config"></a>

* Configure the UHFQA to connect to the PQSC

In [32]:
# Connect UHFQA to the PQSC
uhfqa.enable_qccs_mode()

* Configure the input and output impedances

In [33]:
# Select the input impedance as 50Ω
uhfqa.sigins[0].imp50(1)
uhfqa.sigins[1].imp50(1)
# Select the load impedance as 50Ω
uhfqa.sigouts[0].imp50(1)
uhfqa.sigouts[1].imp50(1)

* Turn on the output channels
* Run the AWG in single mode

In [34]:
# Enable outputs
uhfqa.awg.outputs(["on", "on"])
# Disable rerun
uhfqa.awg.single(True)

* Helper function to generate flattop gaussian envelopes for readout pulses and integrations weights

In [35]:
# Envelope function, with gaussian rise/fall and flattop
def envelope_function(length, rise, a=1.0, sigmas=3):
    x_risefall = np.arange(-rise, rise, 1)
    sigma_wave = rise / sigmas
    risefall_w = np.exp(-((x_risefall) ** 2) / (2 * sigma_wave ** 2))
    length_corr = length - 2 * rise
    flattop = np.ones(length_corr)
    res = a * np.concatenate((risefall_w[:rise], flattop, risefall_w[rise:]), axis=None)
    # plt.plot(res)
    return res

* Quantum Analyzer settings:
    * Bypass deskew and crosstalk matrix since they are not necessary for simulated readout
    * Do not bypass the rotation, it is necessary
    * Set QA delay adjustment to 0
    * Set result source to thresholding

In [36]:
# Bypass all the non-necessary units
uhfqa.qas[0].bypass.deskew(1)
uhfqa.qas[0].bypass.crosstalk(1)
# Don't bypass rotation
uhfqa.qas[0].bypass.rotation(0)
# Set correct QA delay
uhfqa.qa_delay(0)
# Set QA Result source
uhfqa.result_source("Threshold")

* Readout channel settings:
    * Enable weighted integration mode on all readout channels
    * Set Integration length (equal to the readout pulse length)
    * Set threshold level for all channels
    * Set the integrations weights with the envelope generated with the helper function `envelope_function`

In [37]:
# Enable weighted integration mode on all readout channels
uhfqa.enable_readout_channels([i for i in range(num_readout_channels)])
# Set Integration length
uhfqa.integration_length(readout_pulse_length)
# Set threshold level for all channels
for i in range(num_readout_channels):
    uhfqa.channels[i].threshold(0.0)

# Set the integration weights
envelope = envelope_function(readout_pulse_length, rise_fall_length)
for i, freq in enumerate(frequencies):
    uhfqa.channels[i].readout_frequency(freq)
    uhfqa.channels[i].int_weights_envelope(envelope)

# Set rotation coefficients of all channels
# Since UHFQA works with real numbers, the rotation should bring all results on the real axis of the IQ plane
for i, freq in enumerate(frequencies):
    uhfqa.channels[i].rotation(1 - 1j)

* Sequence program which waits for starting trigger from the PQSC, plays the readout pulse, starts quantum analyzer and waits for a while until feedback processing is complete

In [38]:
# Sequence for the UHFQA
uhfqa_program = """\
//Define waveforms for readout
"""

# Add a waveform placeholder for each simulated response
for i in range(num_readouts):
    uhfqa_program += textwrap.dedent(
        f"""\
    wave w{i:d}_I = placeholder($param1$);
    wave w{i:d}_Q = placeholder($param1$);
    """
    )

uhfqa_program += """\

//Start the main program
"""

for i in range(num_readouts):
    uhfqa_program += textwrap.dedent(
        f"""\
    waitZSyncTrigger();  //Wait for start trigger from PQSC
    playWave(1, w{i:d}_I, 2, w{i:d}_Q);  //Play a simulated response
    startQA($param2$, $param3$, $param4$);  //Trigger a readout
    playZero($param5$); //Wait until feedback processing is completed
    
    """
    )

* Configure arguments to use with `startQA` function
    * `trigger_int_unit`: Determines which integration units should be triggered
    * `trigger_qa_input`: Determines if QA Monitor should be triggered
    * `readout_register_address`: Determines the address of the register inside the PQSC to write the results in


In [39]:
# Parameters of UHFQA sequencer
trigger_int_unit = "|".join(f"QA_INT_{i:d}" for i in range(num_readout_channels))
trigger_qa_input = "true"
readout_register_address = register_num

* Setting the sequence parameters and compiling:
    * Use **Custom** sequence type to upload the sequence program.
    * Use **ZSync Trigger** mode to enable HDAWG to receive ZSync triggers
    * Use **custom_params** argument to replace the parameters in the sequence program with the correct numbers and strings

In [40]:
# Set sequence parameters for the UHFQA and compile
uhfqa.awg.set_sequence_params(
    sequence_type="Custom",
    program=uhfqa_program,
    custom_params=[
        readout_pulse_length,
        trigger_int_unit,
        trigger_qa_input,
        readout_register_address,
        wait_samples_uhfqa,
    ],
    trigger_mode="ZSync Trigger",
)

uhfqa.awg.compile()

* Generating the readout pulses and uploading them to the instrument:
    * The for loop cycles through all combinations of the phases [0, 180] to generate simulated responses

In [41]:
# Generate waves
x = np.arange(0, readout_pulse_length, 1)
envelope = envelope_function(readout_pulse_length, rise_fall_length)
waves = []

# Cycle through phases [0, 180] to generate simulated responses
for phs in itertools.product(phases, repeat=num_readout_channels):
    wave_I = np.zeros(readout_pulse_length)
    wave_Q = np.zeros(readout_pulse_length)

    for freq, ph in zip(frequencies, phs):
        wave_I += np.cos(2 * np.pi * freq / SR_UHFQA * x + np.deg2rad(ph))
        wave_Q += np.sin(2 * np.pi * freq / SR_UHFQA * x + np.deg2rad(ph))

    wave_I *= envelope / num_readouts
    wave_Q *= envelope / num_readouts

    waves.append((wave_I, wave_Q))

# Upload waveforms
uhfqa.awg.reset_queue()
for i, wave in enumerate(waves):
    uhfqa.awg.queue_waveform(wave[0], wave[1])
uhfqa.awg.upload_waveforms()

Current length of queue: 1
Current length of queue: 2
Current length of queue: 3
Current length of queue: 4
Upload of 4 waveforms took 0.10871 s


### 8. Main Program <a class="anchor" id="main"></a>

Final check before running the main program:
* Are the connections on ZSync ports fine?

In [42]:
# Check ZSync connections
pqsc.check_zsync_connection(ports=zsync_ports, blocking=True, timeout=30)

Stop all AWGs in case a previous measurement is still going on

In [43]:
# Stop all AWGs
awg_D1.stop()
awg_D2.stop()
awg_AZ.stop()
awg_AX.stop()
uhfqa.awg.stop()

Arm UHFQA for readout, it will wait to be triggered by the AWG sequencer of the UHFQA

In [44]:
# Arm UHFQA readout
uhfqa.arm(length=num_readouts, averages=1)

Start all AWGs, they will wait for triggers from PQSC

In [45]:
# Run all AWGs
uhfqa.awg.run()
awg_D1.run()
awg_D2.run()
awg_AZ.run()
awg_AX.run()

Finally, start sending out triggers from the PQSC

In [46]:
# Start PQSC
pqsc.arm_and_run()